In [1]:
import os
os.chdir("../")
%pwd

'd:\\AI\\NLP\\HandsOn\\Text Summarization'

In [2]:
from dataclasses import dataclass
from pathlib import Path
import torch

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    data_path: Path
    checkpoint: str
    max_length: int
    min_length: int
    output_dir: str
    learning_rate: float
    train_batch_size: int
    eval_batch_size: int
    weight_decay: float
    save_total_limit: int
    num_train_epochs: int
    prefix: str
    push_to_hub: bool
    device: str = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
from TextSummarizer.constants import *
from TextSummarizer.utils.file_utils import *
from TextSummarizer.utils.config_utils import *
from TextSummarizer.utils.lib_utils import *

In [15]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_summarizer_config(self) -> TrainingConfig:
        config = self.params.model

        create_directories([config.root_dir])

        model_summarize_config = TrainingConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            checkpoint = config.checkpoint,
            max_length = config.max_length,
            min_length = config.min_length,
            output_dir = config.output_dir,
            learning_rate = float(config.learning_rate),
            train_batch_size = config.train_batch_size,
            eval_batch_size = config.eval_batch_size,
            weight_decay = config.weight_decay,
            save_total_limit = config.save_total_limit,
            num_train_epochs = config.num_train_epochs,
            push_to_hub = config.push_to_hub,
            device = config.device,
            prefix = config.prefix
        )

        return model_summarize_config

In [5]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer ,AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import pipeline
from datasets import Dataset
from sklearn.model_selection import train_test_split
import torch
from tqdm import tqdm
import evaluate
import nltk
from datasets import DatasetDict, load_dataset, load_from_disk
from pathlib import Path
from transformers import DataCollatorForSeq2Seq

d:\AI\NLP\HandsOn\Text Summarization\textsummarizer-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-12-18 03:36:34,829: INFO: config: PyTorch version 2.5.1+cu121 available.]


In [16]:
class SummarizationModel:
    def __init__(self, config: TrainingConfig):
        logger.info("Initializing SummarizationModel with provided configuration.")
        self.config = config
        self.data_path = self.config.data_path
        self.output_dir = self.config.output_dir
        self.device = self.config.device
        self.checkpoint = self.config.checkpoint
        self.max_length = self.config.max_length
        self.min_length = self.config.min_length
        self.prefix = self.config.prefix
        logger.info("Loading tokenizer and model from checkpoint: %s", self.checkpoint)
        self.tokenizer = AutoTokenizer.from_pretrained(self.checkpoint)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(self.checkpoint).to(self.device)
        logger.info("Tokenizer and model loaded successfully.")
        self.data_collator = DataCollatorForSeq2Seq(
            tokenizer=self.tokenizer,
            model=self.model,
            padding=True,
            max_length=self.max_length
        )
        logger.info("Data collator initialized.")
        self.training_args = Seq2SeqTrainingArguments(
            output_dir=self.output_dir,
            eval_strategy="epoch",
            learning_rate=self.config.learning_rate,
            per_device_train_batch_size=self.config.train_batch_size,
            per_device_eval_batch_size=self.config.eval_batch_size,
            weight_decay=self.config.weight_decay,
            save_total_limit=self.config.save_total_limit,
            num_train_epochs=self.config.num_train_epochs,
            predict_with_generate=True,
            fp16=True,
            push_to_hub=self.config.push_to_hub,
        )
        logger.info("Training arguments configured.")

    def load_data_into_DatasetDict(self) -> DatasetDict:
        logger.info("Loading dataset from path: %s", self.data_path)
        loaded_dataset = load_from_disk(self.data_path)
        logger.info("Dataset loaded successfully with %d splits.", len(loaded_dataset))
        return loaded_dataset

    def compute_metrics(self, eval_pred):
        logger.info("Computing evaluation metrics.")
        rouge = evaluate.load("rouge")
        predictions, labels = eval_pred
        decoded_preds = self.tokenizer.batch_decode(predictions, skip_special_tokens=True)
        labels = np.where(labels != -100, labels, self.tokenizer.pad_token_id)
        decoded_labels = self.tokenizer.batch_decode(labels, skip_special_tokens=True)
        result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
        prediction_lens = [np.count_nonzero(pred != self.tokenizer.pad_token_id) for pred in predictions]
        result["gen_len"] = np.mean(prediction_lens)
        logger.info("Metrics computed: %s", result)
        return {k: round(v, 4) for k, v in result.items()}

    def model_trainer(self, dataset):
        logger.info("Initializing Seq2SeqTrainer for training.")
        trainer = Seq2SeqTrainer(
            model=self.model,
            args=self.training_args,
            train_dataset=dataset["train"],
            eval_dataset=dataset["test"],
            tokenizer=self.tokenizer,
            data_collator=self.data_collator,
            compute_metrics=self.compute_metrics,
        )
        logger.info("Training started.")
        trainer.train()
        logger.info("Training completed.")
        return trainer

    def save_model(self, model):
        logger.info("Saving model to output directory: %s", self.output_dir)
        model.save_model(self.output_dir)
        logger.info("Model saved successfully.")

    def predict(self, text: str) -> str:
        logger.info("Generating summary for input text.")
        inputs = self.tokenizer(
            self.prefix + text,
            max_length=self.max_length,
            truncation=True,
            return_tensors="pt"
        ).to(self.device)
        summary_ids = self.model.generate(
            inputs["input_ids"],
            max_length=self.max_length,
            min_length=self.min_length,
            num_beams=4,
            length_penalty=2.0,
            early_stopping=True
        )
        summary = self.tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        logger.info("Summary generated: %s", summary)
        return summary


In [17]:
try:
    config = ConfigurationManager()
    model_summarizer_config = config.get_model_summarizer_config()
    model_summarizer = SummarizationModel(config=model_summarizer_config)
    data_set = model_summarizer.load_data_into_DatasetDict()
    trained_summarizer = model_summarizer.model_trainer(data_set) 
    model_summarizer.save_model(trained_summarizer)
except Exception as e:
    raise e

[2024-12-18 03:42:16,831: INFO: config_utils: yaml file: config\config.yaml loaded successfully]
[2024-12-18 03:42:16,838: INFO: config_utils: yaml file: params.yaml loaded successfully]
[2024-12-18 03:42:16,840: INFO: file_utils: created directory at: artifacts]
[2024-12-18 03:42:16,842: INFO: file_utils: created directory at: artifacts\data_transformation]
[2024-12-18 03:42:16,843: INFO: 3157424127: Initializing DataStandardization with config]


C:\Users\enay9\AppData\Local\Temp\ipykernel_22220\3157424127.py:46: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
  0%|          | 0/8 [00:00<?, ?it/s]d:\AI\NLP\HandsOn\Text Summarization\textsummarizer-env\Lib\site-packages\transformers\tokenization_utils_base.py:2691: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
 12%|█▎        | 1/8 [00:43<05:01, 43.08s/it]

KeyboardInterrupt: 

In [ ]:
test = "Imagine a world overwhelmed by information, where sifting through endless articles, reports, and data consumes valuable time and energy. AI summarization offers a powerful solution, employing natural language processing to condense lengthy texts into concise summaries. These systems utilize two primary approaches: extractive summarization, selecting key sentences directly from the source, and abstractive summarization, generating new, more human-like summaries that capture the core meaning. This technology finds diverse applications, from streamlining news consumption and accelerating research analysis to optimizing business workflows by summarizing meetings and customer feedback. While challenges remain in handling complex language, nuanced meanings, and ensuring factual accuracy, ongoing research continually improves the fluency, coherence, and contextual understanding of AI-generated summaries. Ultimately, AI summarization promises to revolutionize how we process information, empowering us to access key insights quickly and efficiently, unlocking the potential of knowledge for a more informed future."
summary = SummarizationModel.predict(test)

print(summary)